In [ ]:
!pip install simpy

In [ ]:
import simpy
import random
import time
import math
from statistics import mean, variance
from scipy import stats as st

# Paràmetres
# Cal posar els nivells adequats
ARRIVAL_TIME    = 5 #8,64
QUEUE           = 30 #6
CAM             = 63
TOTAL_TIME      = 86400/2 #86400              # 24 horas
ASSEMBLE        = 40
PORCENTAGE_VEL  = 0.0 #1.0
COSTE_MAQ       = 500
COSTE_LUZ       = 100

# Funcions de probabilitat
def fn_perf():                    # fn_perf() = fn$perf
  #smaller number equals higher priority
  return random.choices([0, 1], weights=[0.2, 0.8])[0]
def fn_xpdis():
  return random.expovariate(1.0)
def fn_destinacio():
  return random.choice([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
def fn_dinero():
  return random.randint(10, 100)

# Recursos globals
class FactoryResources:
  def __init__(self, env):        # simpy.Resource = cola amb servidors
    self.queue = simpy.PriorityResource(env, capacity=QUEUE)
    self.cam = simpy.Resource(env, capacity=CAM)
    self.pto1 = simpy.Resource(env, capacity=1)
    self.pto2 = simpy.Resource(env, capacity=1)

    # Variables estadístiques
    self.env = env

    # variables per calcular benefici
    self.dinero_ganado = 0
    self.dinero_assemble = 0

def assemble_manager(env, store, machine, name):
  while True:
    # Esperar hasta que haya suficientes piezas para ensamblar
    while len(store.items) < ASSEMBLE:
      yield env.timeout(1)

    # Sacar un lote de 40 piezas
    for _ in range(ASSEMBLE):
      yield store.get()

          # Usar el recurso de montaje
    with machine.request() as req:
      yield req
      yield env.timeout(600)  # 600s = 10min

# Decideix a que moll va el paquet
def repart_moll(env, resources, dinero, pto1_store, pto2_store):
    resources.dinero_assemble += dinero;

    if random.random() < 0.5:
        yield pto1_store.put(env.now)
    else:
        yield pto2_store.put(env.now)

# Colas
def normal_process(env, resources, priority, dinero, pto1_store, pto2_store):
    t_start_p = env.now
    with resources.queue.request(priority=priority) as req:
        yield req
        # màquina normal o ràpida
        if random.random() < PORCENTAGE_VEL:
          t = 45;
        else:
          t = 30;

        # triga 30 o 45 segons
        yield env.timeout(t * fn_xpdis())
        # si t = 30, llavors té probabilitat 0.5 de "trencar" el paquet
        # si t = 45, llavors això té probabilitat 0
        if random.random() < (-(t/30) + 1.5):
          dinero -= dinero*0.5

    yield env.process(repart_moll(env, resources, dinero, pto1_store, pto2_store))

def pesa_process(env, resources, priority, dinero, pto1_store, pto2_store):
    with resources.queue.request(priority=priority) as req:
        yield req
        yield env.timeout(45 * fn_xpdis() * 1.5)
    yield env.process(repart_moll(env, resources, dinero, pto1_store, pto2_store))

# Proceso principal de la entidad
def part_process(env, resources, pto1_store, pto2_store):
  priority = fn_perf()
  destinacio = fn_destinacio()
  dinero = fn_dinero()

  if random.random() < 0.01:
    yield env.process(pesa_process(env, resources, priority, dinero, pto1_store, pto2_store))
  else:
    yield env.process(normal_process(env, resources, priority, dinero, pto1_store, pto2_store))

  with resources.cam.request() as req:
    yield req
    yield env.timeout(21600)        # 21600s = 6h

  # acumula els diners obtinguts
  resources.dinero_ganado += resources.dinero_assemble
  resources.dinero_assemble = 0

# Generador d'entitats
def generator(env, resources, pto1_store, pto2_store):
  for _ in range(10000):
    # cada interval de temps ARRIVAL_TIME es genera un nou "paquet"
    yield env.timeout(ARRIVAL_TIME)
    env.process(part_process(env, resources, pto1_store, pto2_store))

In [ ]:
def run_simulation(seed):
    random.seed(seed)
    start = time.time()

    # Crear entorno y recursos
    env = simpy.Environment()
    resources = FactoryResources(env)
    pto1_store = simpy.Store(env)
    pto2_store = simpy.Store(env)

    # Iniciar procesos
    env.process(generator(env, resources, pto1_store, pto2_store))
    env.process(assemble_manager(env, pto1_store, resources.pto1, "PTO1"))
    env.process(assemble_manager(env, pto2_store, resources.pto2, "PTO2"))

    # Ejecutar simulación
    env.run(until=TOTAL_TIME)

    dinero_perdido = (COSTE_LUZ + (QUEUE * COSTE_MAQ)) * (TOTAL_TIME / 3600)

    # Resultat
    end = time.time()
    print(f"Seed {seed}:")
    print(f"  Simulació completada en {end - start:.3f} segons reals.")
    print(f"  Dinero en beneficio:   {resources.dinero_ganado - dinero_perdido:.2f}")
    print(f"  Dinero ganado:  {resources.dinero_ganado:.2f}")
    print(f"  Dinero perdido: {dinero_perdido:.2f}\n")

    return resources.dinero_ganado - dinero_perdido


def run_multiple_simulations():
    results = []
    # modificar el rang per determinar les seeds que es faran servir
    #for seed in range(1200, 11200):
    for seed in range(1, 10):
        total = run_simulation(seed)
        results.append(total)

    n = len(results)
    mean_val = mean(results)
    var_val = variance(results)
    std_err = math.sqrt(var_val / n)

    # t critical for 95% confidence, df = n-1
    t_crit = st.t.ppf(1 - 0.05/2, df=n-1)
    semi_amplitude = t_crit * std_err
    ci_low = mean_val - semi_amplitude
    ci_high = mean_val + semi_amplitude

    print("----- Summary over 10 runs -----")
    print(f"Mean total dinero: {mean_val:.2f}")
    print(f"Variance: {var_val:.2f}")
    print(f"95% CI: [{ci_low:.2f}, {ci_high:.2f}]")
    print(f"Semi-amplitude: {semi_amplitude:.2f}\n\n")

    return {
        "results": results,
        "mean": mean_val,
        "variance": var_val,
        "confidence_interval": (ci_low, ci_high),
        "semi_amplitude": semi_amplitude
    }


In [ ]:
run_multiple_simulations()

Streaming output truncated to the last 5000 lines.

Seed 10368:
  Simulació completada en 0.405 segons reals.
  Dinero en beneficio:   666.00
  Dinero ganado:  181866.00
  Dinero perdido: 181200.00

Seed 10369:
  Simulació completada en 0.586 segons reals.
  Dinero en beneficio:   1980.00
  Dinero ganado:  183180.00
  Dinero perdido: 181200.00

Seed 10370:
  Simulació completada en 0.538 segons reals.
  Dinero en beneficio:   -1928.50
  Dinero ganado:  179271.50
  Dinero perdido: 181200.00

Seed 10371:
  Simulació completada en 0.524 segons reals.
  Dinero en beneficio:   515.00
  Dinero ganado:  181715.00
  Dinero perdido: 181200.00

Seed 10372:
  Simulació completada en 0.413 segons reals.
  Dinero en beneficio:   -2928.00
  Dinero ganado:  178272.00
  Dinero perdido: 181200.00

Seed 10373:
  Simulació completada en 0.570 segons reals.
  Dinero en beneficio:   -1960.00
  Dinero ganado:  179240.00
  Dinero perdido: 181200.00

Seed 10374:
  Simulació completada en 0.531 segons reals.
 

{'results': [-1119.5,
  -916.0,
  -1525.0,
  771.0,
  -1984.0,
  1714.5,
  691.0,
  -1586.0,
  984.0,
  -1819.0,
  -1088.5,
  961.0,
  629.5,
  -512.5,
  1631.0,
  -286.5,
  1109.5,
  780.5,
  1155.0,
  2807.5,
  -2376.0,
  -142.0,
  -483.0,
  2038.0,
  -1563.0,
  -845.5,
  526.5,
  2547.5,
  -24.5,
  1990.0,
  -1030.5,
  -224.0,
  -1494.5,
  1430.5,
  1727.0,
  972.5,
  -1320.0,
  -972.0,
  1758.5,
  3005.5,
  -700.5,
  1124.5,
  -1159.0,
  -75.5,
  185.5,
  -70.0,
  1510.5,
  336.0,
  1144.5,
  2070.0,
  -478.5,
  -910.0,
  2928.0,
  802.0,
  1515.5,
  -2478.5,
  -2124.0,
  808.5,
  -419.0,
  822.0,
  -336.5,
  -684.0,
  1679.5,
  1645.0,
  -3599.0,
  347.0,
  -394.0,
  -1867.5,
  -1596.5,
  4.5,
  674.0,
  -299.0,
  776.5,
  222.0,
  -1295.0,
  -996.5,
  21.0,
  -2570.0,
  2530.0,
  -3126.0,
  1309.5,
  3083.0,
  -1958.5,
  -3142.5,
  42.0,
  160.5,
  -763.5,
  -640.0,
  2067.0,
  -1687.5,
  2535.0,
  803.0,
  -1387.5,
  -1505.0,
  1798.0,
  -2.5,
  -2189.5,
  128.0,
  -212.5,
  -18